# FACE RECOGNITON USING PYTHON (OPENCV)

Their are many cool thing that one can do with this great library (OpenCV). One of which is Face Recognition. This may not be something new but in this repository you will learn how to make a simplest live face recognizer.

Face recognition is the task of identifying an already detected object as a known or unknown face. Often the problem of face recognition is confused with the problem of face detection. Face Recognition on the other hand is to decide if the "face" is someone known, or unknown, using for this purpose a database of faces in order to validate this input face.

I have divided the task of real time facial recognition into the basic steps. The whole project is the combination of these three steps. They are:
		1. DATA SET GENRATION 
		2. TRAINING THE MODEL 
		3. LIVE FACE RECOGNITION


In [ ]:
## IMPORTING MODULES ##
#___________________________________________________________________________________________________________________________

import cv2
import os
import numpy as np
from PIL import Image 

In [ ]:
## PATHS ##
#___________________________________________________________________________________________________________________________

fileDir = os.path.dirname(os.path.realpath('__file__'))
path_of_trainingData = fileDir + r"\trainingData"
path_of_trainingfile = fileDir + r"\trainingdata.yml"
path_of_cascadeClassifier = fileDir + r"\haarcascade_frontalface_default.xml"

In [ ]:
## IMPORTANT INITIALIZATION ##
#___________________________________________________________________________________________________________________________

face_detector = cv2.CascadeClassifier(path_of_cascadeClassifier)
recognizer = cv2.face.LBPHFaceRecognizer_create()
cap = cv2.VideoCapture(0)

## STEP 1 : DataSet Generation

The dataset for face recognition is made in following steps: 
* First of all, permission is granted to web cam and is initialized as **cap**, it starts to record video. Video is the frames or photos running very fast.
* Now in every milisecond of time a frame is captured.
* This frame is then converted into grayscale image as they are easy for comutation. These grayscale are then passed through haar cascade claassifier which is trained to detect faces in frames and provide its location.
* After getting the location of face the faces are croped
* These croped faces are then labeled for that particular person and stored in the folder for training data.
        this is how the face dataset is made now this dataset is send for training.


In [ ]:
## DATASETGENERATOR ##
#___________________________________________________________________________________________________________________________

def datasetGenerator():
    id = input("enter student id:")
    s = 0
    while(True): 
        r, img = cap.read()
        gray = cv2.cvtColor(img,cv2.IMREAD_GRAYSCALE)#COLOR_BGR2GRAY)
        face = face_detector.detectMultiScale(gray,1.3,5)
        for(x,y,w,h) in face:
            cv2.rectangle(gray,(x,y),(x+w,y+h),(0,255,0),5)
            cv2.imwrite(str(path_of_trainingData)+"/"+str(id)+"."+str(s)+".png",gray[y:y+h,x:x+w])
            cv2.imshow("face",gray)
            s=s+1
        if cv2.waitKey(1) & 0xFF== ord("q"):
            break
        if s>100:
            break
    cap.release()
    cv2.destroyAllWindows()

## STEP 2 : TRAINING THE MODEL

* Now in this step the face and there corresponding labels are accessed from training data folder and returned as a numpy array, this work is done by function “getImagesWithID”.
* This function take the path of dataset folder and return the numpy array of image of face and corresponding IDs.
* These are stored in variables and then passed to our recognizing model (LBPH face recognizing model).
* Through train function model is trained and the training data is then stored in an YML file.


In [ ]:
## TRAINING THE MODEL ##
#___________________________________________________________________________________________________________________________

def getImagesWithID(path):
    imagePaths = [os.path.join(path_of_trainingData, f) for f in os.listdir(path_of_trainingData)]   
    faces = []
    IDs = []
    for imagePath in imagePaths:      
        facesImg = Image.open(imagePath).convert('L')
        faceNP = np.array(facesImg, 'uint8')
        ID= int(os.path.split(imagePath)[-1].split(".")[0])
        faces.append(faceNP)
        IDs.append(ID)
        cv2.imshow("Adding faces for traning",faceNP)
        cv2.waitKey(10)
    return np.array(IDs), faces

def trainingTheModel():
    print("TRAINING THE MODEL..................")
    Ids,faces = getImagesWithID(path_of_trainingData)
    cv2.destroyAllWindows()
    recognizer.train(faces,Ids)
    recognizer.save(path_of_trainingfile)
    print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(Ids))))
    cv2.destroyAllWindows()

## STEP 3 : LIVE FACE RECOGNITION

* The data stored in training file is read by LBPH model.
* Again, the web cam is started and start record frame.
* Faces detected in these frame by haar cascade classifier.
* Now these faces are passed to predict function in our model which return the ID and confidence by which it is predicting the given image which has been compared with the face trained.


In [ ]:
## LIVE FACE RECOGNITION ##
#___________________________________________________________________________________________________________________________

def liveFaceRecognition():
    recognizer.read(path_of_trainingfile)
    font = cv2.FONT_HERSHEY_SIMPLEX
    while True:
        ret, im =cap.read()
        gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        faces=face_detector.detectMultiScale(gray, 1.3,5)
        for(x,y,w,h) in faces:
            cv2.rectangle(im,(x,y),(x+w,y+h),(225,0,0),2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
            if(conf<50):
                if(Id==26):
                    Id="gaurav"
            cv2.putText(im,str(Id),(x,y-10),font,0.55,(0,255,0),1)
        cv2.imshow('im',im) 
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## CALLING ALL THREE FUNCTIONS ##
#___________________________________________________________________________________________________________________________

datasetGenerator()
trainingTheModel()
liveFaceRecognition()

##### Here is how the dataset has been generated by the above program:
![](files\Capture.PNG)

##### Training of model on person's face dataset.
![](files/Capture1.PNG)

##### Live face recognition.
![](files/Capture3.PNG)